### CREATE DATAFRAME

In [30]:
from cloudvolume import CloudVolume
from meshparty import skeletonize, trimesh_io
from caveclient import CAVEclient
import trimesh
import numpy as np
import datetime
import networkx as nx
from scipy.sparse import identity
from scipy.spatial import distance_matrix
import scipy 
from tqdm import tqdm
# import aws
import pandas as pd
import csv
import pyembree
import matplotlib.pyplot as plt
import scipy.spatial as spatial
import itertools
import math

In [31]:

orphans = pd.read_csv("/Users/sheeltanna/Desktop/AGT_REPO/campfire/GT_30_Orphans_Spring_2023 - Sheet1.csv")

In [32]:
def my_array(x):
    res = list(map(str.strip, x.split('; ')))
    return res

In [33]:
orphans['endpoints'] = orphans['endpoints'].map(lambda x: list(map(str.strip, x.split('; '))))

In [34]:
## convert from string list to 2-d array
def convert_to_array(row):
    count = 0
    result = []
    for endpoint in row["endpoints"]:
        # endpoint = tuple(map(int, endpoint.split(', ')))
        endpoint = eval(endpoint)
        # print()
        # print(endpoint)
        # print(type(endpoint))
        if(count == 0):
            result = np.array(endpoint)
            count = count + 1
        else:
            result = np.vstack((result, np.array(endpoint)))
            count = count + 1
            #result = np.concatenate(result, list(tuple))
    #check if there was only 1 point, convert to 2-d array:
    # if(type(result) == list):
        
    if(count == 1 and result.size != 0):
        result = result.reshape(1,3)
    return result 

In [35]:
orphans["real_endpoints"] = orphans.apply(convert_to_array, axis = 1)

### TIP FINDER FUNCTIONS


In [36]:
def get_and_process_mesh(root_id):
    datastack_name = "minnie65_phase3_v1"
    client = CAVEclient(datastack_name)
    vol = CloudVolume(
        client.info.segmentation_source(),
        use_https=True,
        progress=False,
        bounded=False,
        fill_missing=True,
        secrets={"token": client.auth.token}
    )
    print("Downloading Mesh")
    mesh = vol.mesh.get(str(root_id))[root_id]
    mesh_obj = trimesh.Trimesh(np.divide(mesh.vertices, np.array([1,1,1])), mesh.faces)
    print("Vertices: ", mesh.vertices.shape[0])

    if mesh_obj.volume > 4000000000000:
        print("TOO BIG, SKIPPING")
        #queue_url_endpoints = sqs.get_or_create_queue("root_ids_functional_dlqueue")

        #entries=sqs.construct_rootid_entries([root_id])

        #sqs.send_batch(queue_url_endpoints, entries)

        return None
    trimesh.repair.fix_normals(mesh_obj)
    mesh_obj.fill_holes()

    return mesh_obj

In [37]:
def get_soma(soma_id:str):
    cave_client = CAVEclient('minnie65_phase3_v1')
    soma = cave_client.materialize.query_table(
        "nucleus_neuron_svm",
        filter_equal_dict={'id':soma_id}
    )
    return soma

In [38]:
def process_mesh_ccs(mesh_obj):
    print("Processing CC's")
    ccs_graph = trimesh.graph.connected_components(mesh_obj.edges)
    ccs_len = [len(c) for c in ccs_graph]

    # Subselect the parts of the mesh that are not inside one another 
    # the other components are an artifact of the soma seg and small unfilled sections
    largest_component = ccs_graph[np.argmax(ccs_len)]
    largest_component_remap = np.arange(ccs_graph[np.argmax(ccs_len)].shape[0])
    face_dict = {largest_component[i]:largest_component_remap[i] for i in range(largest_component.shape[0])}

    new_faces_mask = np.isin(mesh_obj.faces, list(face_dict.keys()))
    new_faces_mask = new_faces_mask[:, 0]*new_faces_mask[:, 1]*new_faces_mask[:, 2]

    new_faces = np.vectorize(face_dict.get)(mesh_obj.faces[new_faces_mask])
    new_faces = new_faces[new_faces[:, 0] != None]
    largest_component_mesh = trimesh.Trimesh(mesh_obj.vertices[largest_component], new_faces)

    all_ids = set(largest_component)
    encapsulated_ids = []

    for i in range(1, len(ccs_graph)):
        n_con = largest_component_mesh.contains(mesh_obj.vertices[ccs_graph[i]])
        if np.sum(n_con) / n_con.shape[0] == 0 and n_con.shape[0] > 50:
            all_ids.update(ccs_graph[i])
        else:
            if len(ccs_graph[i]) < 1000:
                encapsulated_ids.append((np.mean(mesh_obj.vertices[ccs_graph[i]], axis=0)/[4,4,40], len(ccs_graph[i])))
            
    all_component = np.array(list(ccs_graph[np.argmax(ccs_len)]))
    all_component_remap = np.arange(all_component.shape[0])
    face_dict = {all_component[i]:all_component_remap[i] for i in range(all_component.shape[0])}
    new_faces_mask = np.isin(mesh_obj.faces, list(face_dict.keys()))
    new_faces_mask = new_faces_mask[:, 0]*new_faces_mask[:, 1]*new_faces_mask[:, 2]

    new_faces = np.vectorize(face_dict.get)(mesh_obj.faces[new_faces_mask])
    new_faces[new_faces[:, 0] != None]
    
    largest_component_mesh = trimesh.Trimesh(mesh_obj.vertices[all_component], new_faces)
    
    mesh_obj = largest_component_mesh
    return mesh_obj, encapsulated_ids, np.max(ccs_len)

In [39]:
def process_defects(mesh_obj, a=.75):
    bad_edges = trimesh.grouping.group_rows(
        mesh_obj.edges_sorted, require_count=1)
    bad_edges_ind = mesh_obj.edges[bad_edges]
    sparse_edges = mesh_obj.edges_sparse
    xs = list(bad_edges_ind[:, 0]) + list(bad_edges_ind[:, 1]) 
    ys = list(bad_edges_ind[:, 1]) + list(bad_edges_ind[:, 0])
    vs = [1]*bad_edges_ind.shape[0]*2
    bad_inds = scipy.sparse.coo_matrix((vs, (xs, ys)), shape=(mesh_obj.vertices.shape[0], mesh_obj.vertices.shape[0]))
    # Make it symmetrical and add identity so each integrates from itself too, then subtract singleton edges
    # I noticed that the number of asymmetrical edges vs the number of single edges I find from group rows
    # Are close but different. Haven't looked into that yet. Also removing edges 1 hop away from single edges to remove bias towards
    # Holes in the mesh that are caused by mesh construction errors as opposed to segmentation errors
    sparse_edges = mesh_obj.edges_sparse + mesh_obj.edges_sparse.T + identity(mesh_obj.edges_sparse.shape[0]) - sparse_edges.multiply(bad_inds) - bad_inds
    degs = mesh_obj.vertex_degree + 1

    # N_iter is a smoothing parameter here. The loop below smooths the vertex error about the mesh to get more consistent connected regions
    n_iter = 2
    angle_sum = np.array(abs(mesh_obj.face_angles_sparse).sum(axis=1)).flatten()
    defs = (2 * np.pi) - angle_sum

    abs_defs = np.abs(defs)
    abs_defs_i = abs_defs.copy()
    for i in range(n_iter):
        abs_defs_i = sparse_edges.dot(abs_defs_i) / degs
    
    verts_select = np.argwhere((abs_defs_i > a))# & (abs_defs < 2.5))

    edges_mask = np.isin(mesh_obj.edges, verts_select)
    edges_mask[bad_edges] = False
    edges_select = edges_mask[:, 0] * edges_mask[:, 1]
    edges_select = mesh_obj.edges[edges_select]

    G = nx.from_edgelist(edges_select)#f_edge_sub)

    ccs = nx.connected_components(G)
    subgraphs = [G.subgraph(cc).copy() for cc in ccs]

    lens = []
    lengths = []
    for i in tqdm(range(len(subgraphs))):
        ns = np.array(list(subgraphs[i].nodes()))
    #     ns = ns[abs_defs[ns ]]
        l = len(ns)
        if l > 20 and l < 5000:
            lens.append(ns)
            lengths.append(l)
    all_nodes = set()
    for l in lens:
        all_nodes.update(l)
    all_nodes = np.array(list(all_nodes))
    # sharp_pts = mesh_obj.vertices[all_nodes]
    centers = np.array([np.mean(mesh_obj.vertices[list(ppts)],axis=0) for ppts in lens])

    return centers, lens


In [40]:
def process_endpoints(mesh_obj, skel_mp):
    # Process the skeleton to get the endpoints
    interior_cc_mask = set()
    el = nx.from_edgelist(skel_mp.edges)
    comps = list(nx.connected_components(el))
    for c in comps:
        if len(c) < 100:
            n_con = mesh_obj.contains(skel_mp.vertices[list(c)])
            if np.sum(n_con) / n_con.shape[0] > .10:
                interior_cc_mask.update(list(c))
    # Process the skeleton to get the endpoints
    edges = skel_mp.edges.copy()

    edge_mask = ~np.isin(edges, interior_cc_mask)
    edge_mask = edge_mask[:, 0] + edge_mask[:, 1]
    edges = edges[edge_mask]
    edges_flat  = edges.flatten()
    edge_bins = np.bincount(edges_flat) 

    eps = np.squeeze(np.argwhere(edge_bins==1))
    eps_nm = skel_mp.vertices[eps]

    eps_comp = distance_matrix(eps_nm, eps_nm)
    eps_comp[eps_comp == 0] = np.inf
    eps_thresh = np.argwhere(~(np.min(eps_comp, axis=0) < 3000))

    eps = np.squeeze(eps[eps_thresh])
    eps_nm = np.squeeze(eps_nm[eps_thresh])
    return eps, eps_nm

In [41]:
def process_mesh_errors(mesh_obj, centers, eps, eps_nm, lens, skel_mp):
    print("Processing mesh errors")
    # path_to_root_dict = {}
    # for ep in eps:
    #     path_to_root_dict[ep] = skel_mp.path_to_root(ep)
        
    dists_defects = np.zeros(centers.shape[0])
    sizes = np.zeros(centers.shape[0])
    mesh_map = skel_mp.mesh_to_skel_map
    closest_skel_pts = mesh_map[[l[0] for l in lens]]

    # print(centers, eps_nm)

    dist_matrix = distance_matrix(centers, eps_nm)
    ct = 0

    closest_tip = np.zeros((centers.shape[0]))

    for center in tqdm(centers):
    #     skel_pts_dists = np.linalg.norm(skel_mp.vertices - center, axis=1)
    #     ep_pts_dists = np.linalg.norm(eps_nm - center, axis=1)
        
        # closest_skel_pt = closest_skel_pts[ct]
        min_ep = np.inf
        eps_hit = []
        for j, ep in enumerate(eps):
            # if closest_skel_pt in path_to_root_dict[ep]:
            #     eps_hit.append(j)
            eps_hit.append(j)

        if len(eps_hit) == 0:
            dists_defects[ct] = np.inf
            sizes[ct] = np.inf
            ct+=1
            continue
        
        dists = dist_matrix[ct, eps_hit]
    #     print(dists, eps_hit, center / [4,4,40])
        
        amin = np.argmin(dists)
        tip_hit = eps_hit[amin]
        min_dist = dists[amin]
        
        closest_tip[ct] = tip_hit
    #     print(np.argmin(ep_pts_dists), ep_found, eps_nm[np.argmin(ep_pts_dists)]/[4,4,40], eps_nm[j]/[4,4,40], center/[4,4,40])
        dists_defects[ct] = min_dist
        sizes[ct] = len(lens[ct])
        ct+=1
    dists_defects_sub = dists_defects[dists_defects < 5000]
    sizes_sub = sizes[dists_defects < 5000]
    centers_sub = centers[dists_defects < 5000]
    tips_hit_sub = closest_tip[dists_defects < 5000]
    closest_skel_pts_sub = closest_skel_pts[dists_defects < 5000]
    inds_sub = np.arange(centers.shape[0])[dists_defects < 5000]


    # Also ranking each component based on its PCA- if the first component is big enough, the points are mostly linear
    # These point sets seem to be less likely to be true errors
    from sklearn.decomposition import PCA
    pca_vec = np.zeros(inds_sub.shape[0])
    for i in range(inds_sub.shape[0]):
        pca = PCA()#n_components=2)
        pca.fit(mesh_obj.vertices[lens[inds_sub[i]]])

        pca_vec[i] = pca.explained_variance_ratio_[0]

    dists_defects_sub[dists_defects_sub < 4000] = 100
    dists_defects_norm = dists_defects_sub #/ np.max(dists_defects_sub)
    ranks_ep = sizes_sub / dists_defects_norm * (1-pca_vec)
    ranks = sizes_sub**2 * (1-pca_vec)

    #ranks_ep_errors_filt = ranks_ep[ranks_ep > .1]
    centers_ep_send_errors = centers_sub[np.argsort(ranks_ep)][::-1][:20]
    final_mask_eps = np.full(centers_ep_send_errors.shape[0], True)
    tips_hit_send_ep = tips_hit_sub[np.argsort(ranks_ep)][::-1][:20]
    uns, nums = np.unique(tips_hit_send_ep, return_counts=True)

    for un, num in zip(uns, nums):
        if num > 1:
            final_mask_eps[np.argwhere(tips_hit_send_ep == un)[1:]] = False
    centers_errors_ep = centers_ep_send_errors[final_mask_eps]
    centers_errors = centers_sub[np.argsort(ranks)[::-1]][:20]
    return centers_errors, centers_errors_ep

In [42]:
def process_mesh_facets(mesh_obj, skel_mp, eps, eps_nm, facet_area_threshold=30000):
    #changed threshold of facets to 3000 to match changes performed on processed in analysis CELLS
    print("Processing facets")
    #can possibly change param here
    #threshold on size of flat area
    locs = np.argwhere(mesh_obj.facets_area > facet_area_threshold)

    mesh_map = skel_mp.mesh_to_skel_map
    mesh_coords = mesh_obj.vertices[mesh_obj.faces]
    mean_locs = []
    mesh_ind = []
    fs = []
    for l in tqdm(locs):
        fs.append(np.sum(mesh_obj.facets_area[l]))
        fc = mesh_obj.facets[l[0]]
        vert_locs = mesh_coords[fc]
        mean_locs.append(np.mean(vert_locs[:, 0], axis=0))
        mesh_ind.append(fc[0])
    mesh_ind = mesh_obj.faces[mesh_ind][:, 0]
    mean_locs = np.array(mean_locs)
    dists_defects_facets = np.zeros(mean_locs.shape[0])
    mesh_map_facets = skel_mp.mesh_to_skel_map
    closest_skel_pts_facets = mesh_map[[m for m in mesh_ind]]
    dist_matrix_facets = distance_matrix(mean_locs, eps_nm)
    ct = 0

    closest_tip_facets = np.zeros((mean_locs.shape[0]))

    for center in tqdm(mean_locs):

        closest_skel_pt = closest_skel_pts_facets[ct]
        eps_hit = []
        for j, ep in enumerate(eps):
            # if closest_skel_pt in path_to_root_dict[ep]:
            #     eps_hit.append(j)
            eps_hit.append(j)

        if len(eps_hit) == 0:
            dists_defects_facets[ct] = np.inf
            ct+=1
            continue
        
        dists = dist_matrix_facets[ct, eps_hit]
        
        amin = np.argmin(dists)
        tip_hit = eps_hit[amin]
        min_dist = dists[amin]
        
        closest_tip_facets[ct] = tip_hit
        dists_defects_facets[ct] = min_dist
        ct+=1
    dists_defects_sub_facets = dists_defects_facets[dists_defects_facets < 2000]
    sizes_sub_facets = np.array(fs)[dists_defects_facets < 2000]
    mean_locs_facets = mean_locs[dists_defects_facets < 2000]
    tips_hit_sub_facets = closest_tip_facets[dists_defects_facets < 2000]
    closest_skel_pts_sub_facets = closest_skel_pts_facets[dists_defects_facets < 2000]
    inds_sub_facets = np.arange(mean_locs.shape[0])[dists_defects_facets < 2000]
    # ranks_ep_facets = sizes_sub_facets**2 / dists_defects_sub_facets
    #ranks_ep_facets_filt = ranks_ep_facets[ranks_ep_facets > 2e7]
    mean_locs_send_facets = mean_locs_facets
    final_mask_facets = np.full(mean_locs_send_facets.shape[0], True)
    tips_hit_send_facets = tips_hit_sub_facets
    uns, nums = np.unique(tips_hit_send_facets, return_counts=True)

    for un, num in zip(uns, nums):
        if num > 1:
            final_mask_facets[np.argwhere(tips_hit_send_facets == un)[1:]] = False
    facets_send_final = mean_locs_send_facets[final_mask_facets] / [4,4,40]
    return facets_send_final

### TIP FINDER FUNCTION

In [43]:
def error_locs_defects(root_id, soma_id = None, soma_table=None, center_collapse=True):
    #print("START", root_id)

    mesh_obj = get_and_process_mesh(root_id)
    if mesh_obj is None:
        return None
    # SKELETONIZE - if we are just looking for general errors, not errors at endpoints, this can be skipped
    try:
        if soma_table==None:
            soma_table = get_soma(str(soma_id))
        if soma_table[soma_table.id == soma_id].shape[0] > 0:
            center = np.array(soma_table[soma_table.id == soma_id].pt_position)[0] * [4,4,40]
        else:
            center=None
    except:
        center = None
    print("Subselecting largest connected component of mesh")
    mesh_obj, encapsulated_ids, max_verts = process_mesh_ccs(mesh_obj)
    

    skel_mp = skeletonize.skeletonize_mesh(trimesh_io.Mesh(mesh_obj.vertices, 
                                            mesh_obj.faces),
                                            invalidation_d=4000,
                                            shape_function='cone',
                                            collapse_function='branch',
#                                             soma_radius = soma_radius,
                                            soma_pt=center,
                                            smooth_neighborhood=5,
                                             cc_vertex_thresh=max_verts - 10
#                                                     collapse_params = {'dynamic_threshold':True}
                                            )
    print("Skel done")

    # find edges that only occur once..  might be faster to find these in the sparse matrix..
    centers, lens = process_defects(mesh_obj)
    eps, eps_nm = process_endpoints(mesh_obj, skel_mp)

    if len(centers) !=0:
        centers_errors, centers_errors_ep = process_mesh_errors(mesh_obj, centers, eps, eps_nm, lens, skel_mp)
        # ranks_return = np.squeeze(ranks[np.argsort(ranks)[::-1]][:20])
        # ranks_ep_return = np.squeeze(ranks_ep[np.argsort(ranks_ep)][::-1][:20])
    else:
        # Assign placeholder values for each of the variables above.
        centers_errors = np.zeros ((1,3))
        centers_errors_ep = np.zeros ((1,3))
        ranks = np.zeros ((1))
        ranks_ep = np.zeros((1, 3))
        # path_to_root_dict = {}
        # for ep in eps:
        #     path_to_root_dict[ep] = skel_mp.path_to_root(ep)

        ranks_return = 0
        ranks_ep_return = 0


    #if len(centers_errors.shape) > 1 and centers_errors.shape[0] > 0 and len(centers_errors_ep.shape) > 1 and len(centers_errors_ep.shape[0] > 0):
    #    centers_errors = centers_errors[np.min(distance_matrix(centers_errors, centers_errors_ep), axis=1)>1000]
    facets_send_final = process_mesh_facets(mesh_obj, skel_mp, eps, eps_nm)

    errors_send = centers_errors / [4,4,40]
    errors_tips_send = centers_errors_ep / [4,4,40]
    encapsulated_centers = [e[0] for e in encapsulated_ids]
    encapsulated_lens = [e[1] for e in encapsulated_ids]
    sorted_encapsulated_send = np.array(encapsulated_centers)[np.argsort(encapsulated_lens)][::-1]



    return sorted_encapsulated_send, facets_send_final, errors_send, errors_tips_send



### USING TIP GENERATOR

In [44]:
def find_endpoints(row):
    seg_id = row["seg_id"]
    print(seg_id)
    sorted_encapsulated_send, facets_send_final, errors_send, errors_tips_send = error_locs_defects(seg_id)
    #facets_send_final = facets_send_final[facets_send_final != [0,0,0]]
    #errors_tips_send = errors_tips_send[errors_tips_send != [0,0,0]]
    together = np.vstack((facets_send_final, errors_tips_send))
    #should alter "together" if not set together equal to this line
    # print(together)
    # new = together[together != [0,0,0]]
    mask=np.sum(together,axis=1)
    together = together[mask > 0]
    return together

In [45]:
def generate_endpoints(dataframe) :
    dataframe["endpoints_generated"] = dataframe.apply(find_endpoints, axis = 1)
    return dataframe

### ACCURACY FUNCTION

In [46]:
def pred_eps_acc(gt_endpoints, pred_endpoints, threshold):
    # Calculate distances
    dist_matrix = np.array(spatial.distance.cdist(gt_endpoints, pred_endpoints, metric = 'euclidean'))

    # Apply threshold
    dist_matrix[dist_matrix > threshold] = 0

    # Calculating accuracy
    valid_eps = np.count_nonzero(dist_matrix, axis = 1)
    accuracy = np.count_nonzero(valid_eps) / len(gt_endpoints)

    #If more than one valid endpoint found for a single ground truth endpoint, add the other valid endpoints to extra_valid_pairs
    # extra_valid_pairs = []
    # [[extra_valid_pairs.append([gt_endpoints[i], pred_endpoints[index]]) \
    #     for index, j in enumerate(dist_matrix[i]) if j != np.min(dist_matrix[i][dist_matrix[i] != 0]) if j != 0] \
    #         for i in valid_eps if i > 1]

    # return accuracy, extra_valid_pairs
    return accuracy

### TESTING

In [47]:
## TESTING ON THE 30 NEWLY LABELLED ORPHANS

##increase distance threshold
generate_endpoints(orphans)


864691135909994000
Vertices:  7993
Subselecting largest connected component of mesh
Processing CC's


100%|██████████| 7992/7992 [00:00<00:00, 1160454.12it/s]


Skel done


100%|██████████| 26/26 [00:00<00:00, 157589.46it/s]


Processing facets


...

100%|██████████| 48/48 [00:00<00:00, 105461.81it/s]

864691134406233920


Vertices:  1180
Subselecting largest connected component of mesh
Processing CC's


100%|██████████| 1179/1179 [00:00<00:00, 382302.62it/s]
rosetta error: thread_suspend failed


KeyboardInterrupt: 

In [ ]:
#apply function to entire df 
count = 0
acc_array = []
for index, row in orphans.iterrows():
    if (type(row["real_endpoints"])== list and type(row["endpoints_generated"]) == list):
        acc = 1
        print("both empty")
    elif(type(row["endpoints_generated"]) == list and type(row["real_endpoints"]) != list):
        acc = 0
        print("no endpoints generated, but endpoints exist")
    else:
        count = count + 1
        # print("predicted,", row["endpoints_generated"])
        # print()
        # print("real,", row["real_endpoints"])
        # print()
        # print("seg_id", row["seg_id"])
        # print()
        # print(type(row["real_endpoints"]))

        #print(str(count) + ':')
        acc = pred_eps_acc(row["real_endpoints"], row["endpoints_generated"], 250)
        acc_array.append(acc)
        if(acc == 0.0):
            print(row["seg_id"])
        #print(acc)
        # print("NEXT")

864691134472630080
864691134742288112
864691136584897745


In [ ]:
acc_array = np.array(acc_array)
print(acc_array)
print()
print(len(acc_array[acc_array > 0.3]))


###SEVENTY PERCENT OF ORPHANS HAD SOME ENDPOINT FOUND


# print(len(acc_array))
# print(acc_array)
# print(acc_array[])

#smaller mesh volume smaller threshold 
#

[1.         1.         1.         1.         0.5        1.
 0.5        1.         0.         1.         0.         0.75
 1.         1.         0.5        1.         1.         0.
 1.         0.5        0.66666667 1.         0.66666667 0.5
 0.33333333]

22


--Possiby search the 6 perfect ones dependent on task 
--changing invalidation_d resulted in the biggest change in accuracies 
--eps_nm? 


### FINE TUNING

In [ ]:

# indices 2, 5 11

print(orphans["seg_id"][2])
print(orphans["endpoints_generated"][2])

864691135516937065
[[ 77266.35       113085.          20442.45      ]
 [ 79721.25       109333.          20410.425     ]
 [ 79184.6637931  109173.56896552  20430.44741379]]


NOTES FOR SMALL PROCESSES: 

1. index 2 was very small process with two clear flat ends. Skeleton did not reach one flat end and resulted in zero eps_nm and 1 error location which was rightfully filtered out. Changing threshold for eps_nm (not sure if we shoudl do this) found 2 endpoints, but these locations are not correct. They represent the ends of the skeleton. Increasing inval_d did not find the correct endpoints (as exepcted). Decreasing invalidation_d from 5000 to 1500 found one of the flat regions, but endpoint filtered out before getting facets. Endpoint not found if eps_nm threshold returned to 3000. 

2. index 5 also somewhat small, but has 2 flat ends. Skeleton did reach both ends (somewhat), but resulted in zero eps_nm with threshold of 3000. Changed eps_nm threshold to 1500 (not sure if we should do this) and found 2 endpoints (ends of the skeleton). Left invalidation_d quite low at 1500. AFter filtering and scaling, the two endpoints in ep_nm were gone. We should focus on translation from eps_nm to facets. 

3. Index 11 was VERY VERY small and flat. Resonable that skeleton wasn't albe to be processed. 

In [60]:
#starting with Y shaped ONE!!! 
mesh_obj = get_and_process_mesh(864691133906162903)
print("Processing CC's")
ccs_graph = trimesh.graph.connected_components(mesh_obj.edges)
ccs_len = [len(c) for c in ccs_graph]

# Subselect the parts of the mesh that are not inside one another 
# the other components are an artifact of the soma seg and small unfilled sections
largest_component = ccs_graph[np.argmax(ccs_len)]
largest_component_remap = np.arange(ccs_graph[np.argmax(ccs_len)].shape[0])
face_dict = {largest_component[i]:largest_component_remap[i] for i in range(largest_component.shape[0])}

new_faces_mask = np.isin(mesh_obj.faces, list(face_dict.keys()))
new_faces_mask = new_faces_mask[:, 0]*new_faces_mask[:, 1]*new_faces_mask[:, 2]

new_faces = np.vectorize(face_dict.get)(mesh_obj.faces[new_faces_mask])
new_faces = new_faces[new_faces[:, 0] != None]
largest_component_mesh = trimesh.Trimesh(mesh_obj.vertices[largest_component], new_faces)

all_ids = set(largest_component)
encapsulated_ids = []

for i in range(1, len(ccs_graph)):
    n_con = largest_component_mesh.contains(mesh_obj.vertices[ccs_graph[i]])
    if np.sum(n_con) / n_con.shape[0] == 0 and n_con.shape[0] > 50:
        all_ids.update(ccs_graph[i])
    else:
        if len(ccs_graph[i]) < 1000:
            encapsulated_ids.append((np.mean(mesh_obj.vertices[ccs_graph[i]], axis=0)/[4,4,40], len(ccs_graph[i])))
        
all_component = np.array(list(ccs_graph[np.argmax(ccs_len)]))
all_component_remap = np.arange(all_component.shape[0])
face_dict = {all_component[i]:all_component_remap[i] for i in range(all_component.shape[0])}
new_faces_mask = np.isin(mesh_obj.faces, list(face_dict.keys()))
new_faces_mask = new_faces_mask[:, 0]*new_faces_mask[:, 1]*new_faces_mask[:, 2]

new_faces = np.vectorize(face_dict.get)(mesh_obj.faces[new_faces_mask])
new_faces[new_faces[:, 0] != None]

largest_component_mesh = trimesh.Trimesh(mesh_obj.vertices[all_component], new_faces)

mesh_obj = largest_component_mesh

Vertices:  806
Processing CC's


In [61]:
skel_mp = skeletonize.skeletonize_mesh(trimesh_io.Mesh(mesh_obj.vertices, 
                                            mesh_obj.faces),
                                            invalidation_d=5000,
                                            shape_function='cone',
                                            collapse_function='branch',
                                            #soma_radius = soma_radius,
                                            #soma_pt=center,
                                            smooth_neighborhood=5,
                                            #cc_vertex_thresh=max_verts - 10
                                            #collapse_params = {'dynamic_threshold':True}
)

100%|██████████| 805/805 [00:00<00:00, 278743.06it/s]


In [62]:
from meshparty import trimesh_io, trimesh_vtk, skeletonize, mesh_filters
skel_actor = trimesh_vtk.skeleton_actor(skel_mp,
                   edge_property=None,
                   vertex_property=None,
                   vertex_data=None,
                   normalize_property=True,
                   color=(1, 0, 1),
                   line_width=5,
                   opacity=0.7,
                   lut_map=None)

In [63]:
#original = 20
size_cc_threshold = 20
# a is the threshold for the 'jaggedness' of the mesh
# Turn a lower to get more error locations, but these locations will be less jagged
#original: 0.75
a = 0.75

# Removing 'broken' edges - edges in the mesh with only 1 vertex
bad_edges = trimesh.grouping.group_rows(
mesh_obj.edges_sorted, require_count=1)
bad_edges_ind = mesh_obj.edges[bad_edges]
sparse_edges = mesh_obj.edges_sparse
xs = list(bad_edges_ind[:, 0]) + list(bad_edges_ind[:, 1]) 
ys = list(bad_edges_ind[:, 1]) + list(bad_edges_ind[:, 0])
vs = [1]*bad_edges_ind.shape[0]*2

# This averages the jaggedness around each vertex so we can find regions that are gereraly jagged
bad_inds = scipy.sparse.coo_matrix((vs, (xs, ys)), shape=(mesh_obj.vertices.shape[0], mesh_obj.vertices.shape[0]))
# Make it symmetrical and add identity so each integrates from itself too, then subtract singleton edges
# I noticed that the number of asymmetrical edges vs the number of single edges I find from group rows
# Are close but different. Haven't looked into that yet. Also removing edges 1 hop away from single edges to remove bias towards
# Holes in the mesh that are caused by mesh construction errors as opposed to segmentation errors
sparse_edges = mesh_obj.edges_sparse + mesh_obj.edges_sparse.T + identity(mesh_obj.edges_sparse.shape[0]) - sparse_edges.multiply(bad_inds) - bad_inds
degs = mesh_obj.vertex_degree + 1

# N_iter is a smoothing parameter here. The loop below smooths the vertex error about the mesh to get more consistent connected regions
n_iter = 2
angle_sum = np.array(abs(mesh_obj.face_angles_sparse).sum(axis=1)).flatten()
defs = (2 * np.pi) - angle_sum

abs_defs = np.abs(defs)
abs_defs_i = abs_defs.copy()
for i in range(n_iter):
    abs_defs_i = sparse_edges.dot(abs_defs_i) / degs

# Here we are thresholding the error regions to pick only the jagged ones
verts_select = np.argwhere((abs_defs_i > a))# & (abs_defs < 2.5))

edges_mask = np.isin(mesh_obj.edges, verts_select)
edges_mask[bad_edges] = False
edges_select = edges_mask[:, 0] * edges_mask[:, 1]
edges_select = mesh_obj.edges[edges_select]

G = nx.from_edgelist(edges_select)#f_edge_sub)

ccs = nx.connected_components(G)
subgraphs = [G.subgraph(cc).copy() for cc in ccs]

lens = []
lengths = []
for i in tqdm(range(len(subgraphs))):
    ns = np.array(list(subgraphs[i].nodes()))
#     ns = ns[abs_defs[ns ]]
    l = len(ns)
    #manip above 20 (ccs that have jaggedness)
    if l > size_cc_threshold and l < 5000:
        lens.append(ns)
        lengths.append(l)
all_nodes = set()
for l in lens:
    all_nodes.update(l)
all_nodes = np.array(list(all_nodes))
# sharp_pts = mesh_obj.vertices[all_nodes]
centers = np.array([np.mean(mesh_obj.vertices[list(ppts)],axis=0) for ppts in lens])


100%|██████████| 7/7 [00:00<00:00, 17633.71it/s]


In [64]:
interior_cc_mask = set()
el = nx.from_edgelist(skel_mp.edges)
comps = list(nx.connected_components(el))
for c in comps:
    if len(c) < 100:
        n_con = mesh_obj.contains(skel_mp.vertices[list(c)])
        if np.sum(n_con) / n_con.shape[0] > .10:
            interior_cc_mask.update(list(c))
# Process the skeleton to get the endpoints
edges = skel_mp.edges.copy()

edge_mask = ~np.isin(edges, interior_cc_mask)
edge_mask = edge_mask[:, 0] + edge_mask[:, 1]
edges = edges[edge_mask]
edges_flat  = edges.flatten()
edge_bins = np.bincount(edges_flat) 

eps = np.squeeze(np.argwhere(edge_bins==1))
eps_nm = skel_mp.vertices[eps]
print(eps)

eps_comp = distance_matrix(eps_nm, eps_nm)
eps_comp[eps_comp == 0] = np.inf
#MODIFIED (actually changed back to 3000)
eps_thresh = np.argwhere(~(np.min(eps_comp, axis=0) < 3000))

eps = np.squeeze(eps[eps_thresh])
eps_nm = np.squeeze(eps_nm[eps_thresh])
print()
print(eps_nm)
print(eps_nm.shape)
print(len(eps))
print(eps.shape)

[28 30]

[[489195. 942627. 851130.]
 [489321. 943971. 846426.]]
(2, 3)
2
(2,)


In [65]:
locs

array([[  8],
       [ 87],
       [ 89],
       [133],
       [160],
       [192],
       [216],
       [220],
       [221],
       [229],
       [236],
       [238],
       [240]])

In [66]:
eps

array([28, 30])

In [ ]:
from numbers import Number
isinstance(eps, Number)

False

In [ ]:
eps.size

3

In [70]:
#are of flat region
#try to decrease/increase
facet_area_threshold = 30000
#threshold distance from tip to jagged error or a facet
#try increasing to get more
#this threshold impacts the facets
error_distance_threshold = 2000


if eps.size == 1:
        print("hello there")
        #eps_nm should therefore be reshaped
        eps_nm = eps_nm.reshape((1,3))


# path_to_root_dict = {}
# for ep in eps:
#     #doesn't make sense for orphans to include 
#     path_to_root_dict[ep] = skel_mp.path_to_root(ep)


print("Processing facets")
#can possibly change param here
#th
# reshold on size of flat area
locs = np.argwhere(mesh_obj.facets_area > facet_area_threshold)

mesh_map = skel_mp.mesh_to_skel_map
mesh_coords = mesh_obj.vertices[mesh_obj.faces]
mean_locs = []
mesh_ind = []
fs = []
for l in tqdm(locs):
    #mesh coords is each vertex coordinate 
    #need to index normal vector
    fs.append(np.sum(mesh_obj.facets_area[l]))
    fc = mesh_obj.facets[l[0]]
    # print(np.sum(mesh_obj.facets_area[l]))
    vert_locs = mesh_coords[fc]
    mean_locs.append(np.mean(vert_locs[:, 0], axis=0))
    mesh_ind.append(fc[0])
mesh_ind = mesh_obj.faces[mesh_ind][:, 0]
mean_locs = np.array(mean_locs)
dists_defects_facets = np.zeros(mean_locs.shape[0])
mesh_map_facets = skel_mp.mesh_to_skel_map
closest_skel_pts_facets = mesh_map[[m for m in mesh_ind]]
print(mean_locs)
#distance from every facet to each skel endpoint
dist_matrix_facets = distance_matrix(mean_locs, eps_nm)
print(dist_matrix_facets)
#represents index of the current facet
ct = 0

closest_tip_facets = np.zeros((mean_locs.shape[0]))

for center in tqdm(mean_locs):
    #mapping current facet to closet point on skeleton 
    #checking if the skel point in in path from ep to some
    #doesn't make sense for orphans
    closest_skel_pt = closest_skel_pts_facets[ct]
    eps_hit = []
    if eps.size == 1:
        eps_hit.append(eps)
    else:
        for j, ep in enumerate(eps):
            # if closest_skel_pt in path_to_root_dict[ep]:
            #     eps_hit.append(j)
            eps_hit.append(j)


    #distance from every error(facets and jagged) to every endpoint
    if(eps.size == 1):
         dists = dist_matrix_facets[ct]
    else:
         dists = dist_matrix_facets[ct, eps_hit]
    
    #index of closest endpoint for current facet
    amin = np.argmin(dists)
    # print(amin)
    #The code retrieves the index of the closest endpoint in the original eps array
    tip_hit = eps_hit[amin]
    # print(tip_hit)
    # print()
    min_dist = dists[amin]
    # print(min_dist)
    # print()
    

    closest_tip_facets[ct] = tip_hit
    dists_defects_facets[ct] = min_dist
    ct+=1

#printing out closest_tip_facets
print(closest_tip_facets)
print(dists_defects_facets)

#filtered distances
dists_defects_sub_facets = dists_defects_facets[dists_defects_facets < error_distance_threshold]
sizes_sub_facets = np.array(fs)[dists_defects_facets < error_distance_threshold]
#look at mean locs: getting vector of booleans: see if anything getting filtered
#if large distance set threshold below distance
mean_locs_facets = mean_locs[dists_defects_facets < error_distance_threshold]



#ADDED//
#change back to np.inf???
tips_hit_sub_facets = closest_tip_facets[dists_defects_facets < error_distance_threshold]
closest_skel_pts_sub_facets = closest_skel_pts_facets[dists_defects_facets < error_distance_threshold]
inds_sub_facets = np.arange(mean_locs.shape[0])[dists_defects_facets < error_distance_threshold]
#END

#change rank function, possibly change sizes, add a threshold for all

#NEW CODE!!!!! 
ranks_ep_facets = 1 / dists_defects_sub_facets
#(no longer including ranking)
mean_locs_send_facets = mean_locs_facets
#mean_locs_send_facets = mean_locs_facets[np.argsort(ranks_ep_facets)][::-1][:20]
final_mask_facets = np.full(mean_locs_send_facets.shape[0], True)


#OLD code
# ranks_ep_facets = sizes_sub_facets**2 / dists_defects_sub_facets
# mean_locs_send_facets = mean_locs_facets[np.argsort(ranks_ep_facets)][::-1][:20]
# final_mask_facets = np.full(mean_locs_send_facets.shape[0], True)
# tips_hit_send_facets = tips_hit_sub_facets[np.argsort(ranks_ep_facets)][::-1][:20]
# uns, nums = np.unique(tips_hit_send_facets, return_counts=True)
# for un, num in zip(uns, nums):
#         if num > 1:
#             final_mask_facets[np.argwhere(tips_hit_send_facets == un)[1:]] = False


#ADDED in this as well: this code is filtering out duplicates, comment out for now
#tips_hit_send_facets = tips_hit_sub_facets
# got rid of ranking
# HELPS 
tips_hit_send_facets = tips_hit_sub_facets
uns, nums = np.unique(tips_hit_send_facets, return_counts=True)
for un, num in zip(uns, nums):
    if num > 1:
        final_mask_facets[np.argwhere(tips_hit_send_facets == un)[1:]] = False
# #END


# This is the merged facets variable!!
facets_send_final = mean_locs_send_facets[final_mask_facets] / [4,4,40]
#facets_send_final = mean_locs_send_facets/ [4,4,40]
print(facets_send_final)


Processing facets


0it [00:00, ?it/s]

[]


ValueError: not enough values to unpack (expected 2, got 1)

In [72]:
mean_locs.size

0

In [68]:
mean_locs

array([], dtype=float64)

In [69]:
eps_nm

TrackedArray([[489195., 942627., 851130.],
              [489321., 943971., 846426.]])

In [55]:
locs

array([[  8],
       [ 87],
       [ 89],
       [133],
       [160],
       [192],
       [216],
       [220],
       [221],
       [229],
       [236],
       [238],
       [240]])

In [ ]:
eps

array([124, 235, 248])

In [ ]:
mesh_obj.facets_area

array([ 4687.89430342, 14125.77452744, 14032.53407265, ...,
       14850.63816811, 12728.02706628,  7056.        ])

In [ ]:
mean_locs

array([[400170.75      , 690637.5       , 752535.        ],
       [403739.        , 697606.        , 748377.        ],
       [403710.17647059, 697547.11764706, 748545.        ],
       [407671.6       , 697635.4       , 747894.        ],
       [405925.96153846, 699199.84615385, 745416.        ],
       [406419.77419355, 697145.12903226, 746697.        ],
       [406735.        , 697083.8       , 746025.        ],
       [406486.5       , 697404.75      , 745983.        ],
       [406105.875     , 697719.75      , 746613.        ],
       [405065.5       , 697917.5       , 747789.        ],
       [403350.        , 695589.        , 749133.        ],
       [403086.        , 695520.        , 749217.        ],
       [403368.        , 695486.4       , 749427.        ],
       [400943.66666667, 694288.        , 750855.        ],
       [400816.5       , 694540.        , 750183.        ],
       [401236.5       , 695131.5       , 749658.        ],
       [401608.2       , 695101.26      

In [ ]:
closest_skel_pts_facets

array([272, 364, 364, 314,  53,  43,  43,  43,  43, 314, 214, 220, 213,
       175, 175, 175, 175, 175, 175, 175, 169, 262, 228,   1,  64,  64,
        64,  85, 135, 114])

In [ ]:
dists = dist_matrix_facets[ct, eps_hit]
dist_matrix_facets

IndexError: index 30 is out of bounds for axis 0 with size 30

: 

In [ ]:
eps

array([ 63, 103])

In [ ]:
eps_nm / [4,4,40]

TrackedArray([[351109.5  , 143466.75 ,  15149.4  ],
              [348705.   , 145225.5  ,  14892.675]])

In [ ]:
dists

array([ 3026.55173589,  4811.43914125,   519.60043303, 11271.84091486,
        8842.88555902])

In [ ]:
dist_matrix_facets

array([[ 2477.23484515, 17655.92736133],
       [ 2404.4789082 , 17977.80085606],
       [ 4223.00943451, 14810.92911046],
       [ 3620.90893407, 15574.12721114],
       [ 2638.25908508, 16414.83356602],
       [ 2584.10718458, 16757.06704845],
       [ 2559.96050995, 17998.55364502],
       [ 2268.72245009, 18801.92297494],
       [17985.97664738,   459.49228503],
       [ 6628.9186901 , 11895.14224379],
       [ 6614.21024092, 11942.49601679],
       [ 6213.394161  , 12347.27180392],
       [ 6618.2156793 , 11927.30869802],
       [ 6329.24365545, 12227.14317615],
       [ 7276.96241573, 11234.25297917],
       [ 8063.81361392, 10455.00645624],
       [ 9511.28372608,  9050.42767507],
       [ 9474.39807439,  9030.37112978],
       [11035.44902802,  7526.92302671],
       [12115.54137668,  6379.22658713],
       [13902.89054214,  4828.81339784],
       [12993.197622  ,  5576.87461258],
       [13409.35896963,  5198.84127395],
       [16699.83913096,  1984.85490654],
       [16268.75

In [ ]:
closest_skel_pts_facets

array([ 40,  40,  12,  87,  89,  88,  36,  40, 128,  21,  21,  32,  32,
        31,  26, 159, 174, 174, 166, 106, 111, 113, 110, 149, 132, 132])

In [ ]:
closest_tip_facets

array([2., 2.])

In [ ]:
closest_skel_pts_facets

array([ 40,  40,  12,  87,  89,  88,  36,  40, 128,  21,  21,  32,  32,
        31,  26, 159, 174, 174, 166, 106, 111, 113, 110, 149, 132, 132])

In [ ]:
tip_hit

0

In [ ]:
dists 

array([16125.57229992])

In [ ]:
mean_locs

array([[316822.08510638, 436990.78723404, 817341.        ],
       [316950.9       , 436697.1       , 817257.        ],
       [315507.24861878, 439514.56906077, 817341.        ],
       [315775.32      , 438889.92      , 816018.        ],
       [316641.        , 438390.        , 816018.        ],
       [316529.0625    , 437897.25      , 816459.        ],
       [316685.25      , 436621.5       , 816018.        ],
       [317531.66666667, 436067.33333333, 817341.        ],
       [309065.4       , 452340.        , 817698.        ],
       [314727.        , 442428.        , 816942.        ],
       [314736.33333333, 442358.        , 817341.        ],
       [314839.        , 441980.        , 816816.        ],
       [314650.        , 442360.33333333, 816774.        ],
       [314867.        , 442179.5       , 816375.        ],
       [314433.        , 443052.        , 816627.        ],
       [314067.        , 443772.        , 816417.        ],
       [313703.73770492, 445291.        

In [ ]:
dists_defects_facets.shape
mean_locs.shape

(26, 3)

In [ ]:
# skel Distance threshold to the defects (the jagged ones)
#BUT this impacts the center_errors
#try to increase to get more points if not finding flat regions
error_distance_threshold_defects = 10000 #1000


dists_defects = np.zeros(centers.shape[0])


sizes = np.zeros(centers.shape[0])
mesh_map = skel_mp.mesh_to_skel_map
closest_skel_pts = mesh_map[[l[0] for l in lens]]

# print(centers, eps_nm)

print("centers " + str(type(centers)))
print("eps_nm " + str(type(eps_nm)))
print(len(centers))

if len(centers) == 0:
    centers_errors_ep = [0,0,0]
    

else:
    #distance matrix of center and endpoints before merging right???
    dist_matrix = distance_matrix(centers, eps_nm)
    ct = 0

    closest_tip = np.zeros((centers.shape[0]))

    for center in tqdm(centers):
    #     skel_pts_dists = np.linalg.norm(skel_mp.vertices - center, axis=1)
    #     ep_pts_dists = np.linalg.norm(eps_nm - center, axis=1)
        
        closest_skel_pt = closest_skel_pts[ct]
        min_ep = np.inf
        eps_hit = []
        for j, ep in enumerate(eps):
            #path to root doesn't make sense, let us compare all eps
            # if closest_skel_pt in path_to_root_dict[ep]:
            #     eps_hit.append(j)
            eps_hit.append(j)
        #doesn't make sense anymore
        if len(eps_hit) == 0:
            dists_defects[ct] = np.inf
            sizes[ct] = np.inf
            ct+=1
            continue
        
        dists = dist_matrix[ct, eps_hit]
        #print(dists)
    #     print(dists, eps_hit, center / [4,4,40])
        
        amin = np.argmin(dists)
        tip_hit = eps_hit[amin]
        min_dist = dists[amin]
        
        closest_tip[ct] = tip_hit
    #     print(np.argmin(ep_pts_dists), ep_found, eps_nm[np.argmin(ep_pts_dists)]/[4,4,40], eps_nm[j]/[4,4,40], center/[4,4,40])
        dists_defects[ct] = min_dist
        # print(dists_defects)
        # print()
        sizes[ct] = len(lens[ct])
        ct+=1

        
    dists_defects_sub = dists_defects[dists_defects < error_distance_threshold_defects]
    sizes_sub = sizes[dists_defects < error_distance_threshold_defects]

    centers_sub = centers[dists_defects < error_distance_threshold_defects]
    # print(centers_sub)
    # print()

    tips_hit_sub = closest_tip[dists_defects < error_distance_threshold_defects]
    closest_skel_pts_sub = closest_skel_pts[dists_defects < error_distance_threshold_defects]
    inds_sub = np.arange(centers.shape[0])[dists_defects < error_distance_threshold_defects]


    # Also ranking each component based on its PCA- if the first component is big enough, the points are mostly linear
    # These point sets seem to be less likely to be true errors
    from sklearn.decomposition import PCA
    pca_vec = np.zeros(inds_sub.shape[0])
    for i in range(inds_sub.shape[0]):
        pca = PCA()#n_components=2)
        pca.fit(mesh_obj.vertices[lens[inds_sub[i]]])

        pca_vec[i] = pca.explained_variance_ratio_[0]

    # dists_defects_sub[dists_defects_sub < 4000] = 100
    dists_defects_norm = dists_defects_sub #/ np.max(dists_defects_sub)
    ranks_ep = sizes_sub / dists_defects_norm * (1-pca_vec)
    # ranks = sizes_sub**2 * (1-pca_vec)

    #ranks_ep_errors_filt = ranks_ep[ranks_ep > .1]
    #this will overreport errors

    # centers_ep_thresholded = centers_sub[ranks_ep < ranks_threshold]
    # centers_thresholded = centers_sub[ranks < ranks_threshold]


    #top 20 what???
    centers_ep_send_errors = centers_sub[np.argsort(ranks_ep)][::-1][:20]

    # print(centers_ep_send_errors)
    # print()
    
    final_mask_eps = np.full(centers_ep_send_errors.shape[0], True)
    tips_hit_send_ep = tips_hit_sub[np.argsort(ranks_ep)][::-1][:20]
    uns, nums = np.unique(tips_hit_send_ep, return_counts=True)

    for un, num in zip(uns, nums):
        if num > 1:
            final_mask_eps[np.argwhere(tips_hit_send_ep == un)[1:]] = False

    # Centers_errors_ep are the error locations, ranked by how close they are to an endpoint
    centers_errors_ep = centers_ep_send_errors[final_mask_eps]
    print(centers_errors_ep)
    # This one doesn't care if we are close to an endpoint
    # centers_errors = centers_sub[np.argsort(ranks)[::-1]][:20]



centers <class 'numpy.ndarray'>
eps_nm <class 'trimesh.caching.TrackedArray'>
11


100%|██████████| 11/11 [00:00<00:00, 5410.74it/s]

[[402859.3902439  694017.73170732 741139.17073171]
 [410665.5        702794.02941176 739061.64705882]]


### VISUALIZING

In [ ]:
# centers_errors_ep is output of jagged errors merged with skel endpoints
# centers is output of jagged errors
# facets_send_final is output of flat errors merged with skel endpoints
# mean_locs is the output of flat errors
# eps_nm is the skel endpoints

#VISUALZING HERE
# syn_actor2 purple 
# mesh object is gray
# syn_actor = trimesh_vtk.point_cloud_actor(eps_nm, size=100, color=(0.0, 0.0, 0.9))
# syn_actor2 = trimesh_vtk.point_cloud_actor(centers, size=100, color=(0.9, 0.2, 0.9))
# mesh_actor = trimesh_vtk.mesh_actor(mesh_obj, opacity=1, color=(0.7, 0.7, 0.7))
# trimesh_vtk.render_actors([mesh_actor, skel_actor, syn_actor2, syn_actor])

#VISUALZING HERE 
#also visualize facets_send_final
# syn_actor = trimesh_vtk.point_cloud_actor(eps_nm, size=100, color=(0.0, 0.0, 0.9))
#these will become errors_tips_send
# purple point shoudl center_errors_ep


# syn_actor4 = trimesh_vtk.point_cloud_actor(mean_locs, opacity = 0.75, size=200, color=(0.9, 0.2, 0.9))
# #syn_actor = trimesh_vtk.point_cloud_actor(eps_nm, opacity = 0.75, size=200, color=(0.9, 0.2, 0.9))
# # unscaled_facets_send_final = facets_send_final * [4,4,40]
# # #PURPLE
# syn_actor2 = trimesh_vtk.point_cloud_actor(centers_errors_ep, size=800, color=(0.9, 0.2, 0.9))
# # #GREEN
# syn_actor3 = trimesh_vtk.point_cloud_actor(unscaled_facets_send_final, size=800, color=(0.0, 0.9, 0.0))
# #GRAY
mesh_actor = trimesh_vtk.mesh_actor(mesh_obj, opacity=1, color=(0.7, 0.7, 0.7))
#SKEL = PINK
trimesh_vtk.render_actors([mesh_actor, skel_actor])


# endpoints_gen = facets_send_final 

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x7fec58008a00) at 0x7feca14450a0>

In [ ]:
facets_send_final

array([[350628.6 , 143806.95,  15044.4 ]])

In [ ]:
together = np.vstack((facets_send_final, centers_errors_ep/[4,4,40]))

In [ ]:
real = [[401602, 224623, 23991],[405273, 226312, 23618],[409204, 219553, 23892]]
acc = pred_eps_acc(real, together, 300)
print(acc)
print(together)
print()
print(real)

0.0
[[ 79205.5212766  109247.69680851  20433.525     ]
 [ 79237.725      109174.275       20431.425     ]
 [ 78876.8121547  109878.64226519  20433.525     ]
 [ 78943.83       109722.48        20400.45      ]
 [ 79160.25       109597.5         20400.45      ]
 [ 79132.265625   109474.3125      20411.475     ]
 [ 79171.3125     109155.375       20400.45      ]
 [ 79382.91666667 109016.83333333  20433.525     ]
 [ 78681.75       110607.          20423.55      ]
 [ 78684.08333333 110589.5         20433.525     ]
 [ 78709.75       110495.          20420.4       ]
 [ 78662.5        110590.08333333  20419.35      ]
 [ 78716.75       110544.875       20409.375     ]
 [ 78608.25       110763.          20415.675     ]
 [ 78516.75       110943.          20410.425     ]
 [ 78425.93442623 111322.75        20400.45      ]
 [ 78404.52717391 111299.08695652  20408.325     ]
 [ 79759.43181818 109328.38636364  20413.69431818]]

[[401602, 224623, 23991], [405273, 226312, 23618], [409204, 219553, 23892]]


In [ ]:

for point in real: 
    for other_point in together: 
        dist = math.sqrt((point[0]-other_point[0])**2) + (point[1] - other_point[1])**2 + (point[2]-other_point[2])**2)
        print(dist)
        print()

SyntaxError: unmatched ')' (3121358080.py, line 3)

### BAD AND GOOD PROCESS 


In [ ]:
from IPython.display import Image
###PROCESS THAT it DOESN'T DO WELL ON
Image('/Users/sheeltanna/Desktop/AGT_REPO/campfire/Small_Orphan_Image.png')

In [ ]:
from IPython.display import Image
###PROCESS THAT it DOESN'T DO WELL ON
Image('/Users/sheeltanna/Desktop/AGT_REPO/campfire/Screen Shot 2023-04-05 at 11.10.49 PM.png')

In [ ]:
#Getting the predicted endpoints 
sorted_encapsulated_send, facets_send_final, errors_send, errors_tips_send, dummy, dummy2, dummy3 = error_locs_defects(864691135909994000)

In [ ]:
facets_send_final

In [ ]:
#getting the accuracy + seeing the predicted vs real endpoints
together = np.vstack((facets_send_final, errors_tips_send))
    #should alter "together" if not set together equal to this line
    # print(together)
    # new = together[together != [0,0,0]]
mask=np.sum(together,axis=1)
together = together[mask > 0]
print("predicted")
print(together)
print()
print("real")
print(orphans["real_endpoints"][1])
print()
print("accurcy")
acc = pred_eps_acc(orphans["real_endpoints"][1], together, 200)
print(acc)


In [ ]:
print("before scaling of endpoints")
print(eps_nm)
print()
print("after scaling of endpoints")
print(facets_send_final)
print()
print("before scaling of errors")
print(centers)
print()
print("after scaling of errors")
print(errors_tips_send)

4 Hyperparams now
observe changes on handful of orphans (visualize mesh)
one notebook, clean calls, display figure
have all of our updated params 
break down the endpoints we do find 
show example of orphan 
matplotlib + images+ illustrator etc 

DUE: END OF SEMESTER